In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 20.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=dd745dd7b5d0aeae932faed121328e4b0c1ba473d8a36ec86a05a6f64f2939c8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
cur_path = "/content/drive/My Drive/big_data/final_project/"

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('final').getOrCreate()

In [6]:
data = spark.read.csv(cur_path+"data.csv",inferSchema=True,header=True)

In [7]:
data.show()

+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+----------------+--------+-------+
|               date|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|              street|            city|statezip|country|
+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+----------------+--------+-------+
|2014-05-02 00:00:00| 313000.0|     3.0|      1.5|       1340|    7912|   1.5|         0|   0|        3|      1340|            0|    1955|        2005|18810 Densmore Ave N|       Shoreline|WA 98133|    USA|
|2014-05-02 00:00:00|2384000.0|     5.0|      2.5|       3650|    9050|   2.0|         0|   4|        5|      3370|          280|    1921|           0|     709 W Blaine St|

In [8]:
output=spark.read.csv(cur_path+"output.csv",inferSchema=True,header=True)

In [9]:
output.show()

+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+----------------+--------+-------+
|               date|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|              street|            city|statezip|country|
+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+----------------+--------+-------+
|2014-05-02 00:00:00| 313000.0|     3.0|      1.5|       1340|    7912|   1.5|         0|   0|        3|      1340|            0|    1955|        2005|18810 Densmore Ave N|       Shoreline|WA 98133|    USA|
|2014-05-02 00:00:00|2384000.0|     5.0|      2.5|       3650|    9050|   2.0|         0|   4|        5|      3370|          280|    1921|           0|     709 W Blaine St|

In [10]:
print((data.count(), len(data.columns)))

(4600, 18)


In [11]:
print((output.count(),len(output.columns)))

(4600, 18)


In [12]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [13]:
print(data.select('city').distinct().count())
print(data.select('street').distinct().count(), 'multiple of same street address???')
print(data.select('statezip').distinct().count())
print(data.select('country').distinct().count())

44
4525 multiple of same street address???
77
1


In [14]:
data.groupBy('street').count().orderBy('count',ascending=False).show() #### ???????????????????????

+--------------------+-----+
|              street|count|
+--------------------+-----+
|2520 Mulberry Wal...|    4|
|2500 Mulberry Wal...|    3|
|  13800 453rd Ave SE|    2|
|  Burke-Gilman Trail|    2|
| 35229 SE Terrace St|    2|
|     6008 8th Ave NE|    2|
|     1610 N 185th St|    2|
|    7490 85th Ave SE|    2|
|    13820 NE 72nd Pl|    2|
|    9413 34th Ave SW|    2|
|      353 N 105th St|    2|
|   8434 W Mercer Way|    2|
|       513 N 46th St|    2|
|      323 25th Ave S|    2|
|     2050 14th Ave W|    2|
|  13420 121st Ave NE|    2|
|       611 N 46th St|    2|
| 2358 Fairview Ave E|    2|
|   4507 Linden Ave N|    2|
|      820 NW 95th St|    2|
+--------------------+-----+
only showing top 20 rows



In [15]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show() # NO NA's

+----+-----+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+------+----+--------+-------+
|date|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|street|city|statezip|country|
+----+-----+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+------+----+--------+-------+
|   0|    0|       0|        0|          0|       0|     0|         0|   0|        0|         0|            0|       0|           0|     0|   0|       0|      0|
+----+-----+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+------+----+--------+-------+



In [16]:
data.distinct().count()

4600

In [17]:
data.groupBy('price').count().orderBy('price').show(n=20)


+-------------+-----+
|        price|count|
+-------------+-----+
|          0.0|   49|
|       7800.0|    1|
|      80000.0|    1|
|      83000.0|    1|
|      83300.0|    2|
|      84350.0|    1|
|      87500.0|    1|
|      90000.0|    2|
|     100000.0|    4|
|     102500.0|    1|
|     107500.0|    1|
|108333.333333|    1|
|     110000.0|    2|
|     110700.0|    1|
|     115000.0|    1|
|117833.333333|    1|
|     119500.0|    1|
|     120000.0|    1|
|     120750.0|    1|
|     124740.0|    1|
+-------------+-----+
only showing top 20 rows



In [18]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import OneHotEncoder, StringIndexer, MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression


In [19]:
tokenizer = Tokenizer(inputCol="street", outputCol="tokenized_street")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="tokenized_features")
encoder_city = OneHotEncoder(
    inputCols=["qualificationcity"],  
    outputCols=["city_vector"]
)
encoder_zip = OneHotEncoder(
    inputCols=["qualificationzip"],  
    outputCols=["statezip_vector"]
)
qualification_indexer_city = StringIndexer(inputCol="city", outputCol="qualificationcity")
qualification_indexer_zip = StringIndexer(inputCol="statezip", outputCol="qualificationzip")

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

vectorAssembler = VectorAssembler(inputCols=['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','sqft_above','sqft_basement','yr_built','yr_renovated','qualificationcity','qualificationzip','tokenized_features'], outputCol='features')
lr = LinearRegression(labelCol='price')

pipeline = Pipeline(stages=[tokenizer, hashingTF,qualification_indexer_city,qualification_indexer_zip, encoder_city, encoder_zip, vectorAssembler, lr])

In [20]:
final_data = data.select(['bedrooms','price','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','sqft_above','sqft_basement','yr_built','yr_renovated','street','city','statezip'])
final_data=final_data.where("price!=0")


In [21]:
train_data, test_data = final_data.randomSplit(weights=[0.7,0.3],seed=74)

In [22]:
train_data.show()

+--------+---------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+
|bedrooms|    price|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|              street|     city|statezip|
+--------+---------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+
|     0.0|1095000.0|      0.0|       3064|    4764|   3.5|         0|   2|        3|      3064|            0|    1990|        2009|       814 E Howe St|  Seattle|WA 98102|
|     0.0|1295648.0|      0.0|       4810|   28008|   2.0|         0|   0|        3|      4810|            0|    1990|        2009|    20418 NE 64th Pl|  Redmond|WA 98053|
|     1.0|  80000.0|     0.75|        430|    5050|   1.0|         0|   0|        2|       430|            0|    1912|           0|32215 E R

In [23]:
df_transformed = pipeline.fit(final_data)


In [24]:
prediction=df_transformed.transform(test_data)

In [25]:
prediction.show()

+--------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+--------------------+--------------------+-----------------+----------------+---------------+---------------+--------------------+------------------+
|bedrooms|   price|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|sqft_above|sqft_basement|yr_built|yr_renovated|              street|     city|statezip|    tokenized_street|  tokenized_features|qualificationcity|qualificationzip|    city_vector|statezip_vector|            features|        prediction|
+--------+--------+---------+-----------+--------+------+----------+----+---------+----------+-------------+--------+------------+--------------------+---------+--------+--------------------+--------------------+-----------------+----------------+---------------+---------------+--------------------+------------------+
|     1.0|115000.0|      2.0|       1150

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='price')
evaluator.setPredictionCol("prediction")
evaluator.evaluate(prediction, {evaluator.metricName: "r2"})
evaluator.evaluate(prediction, {evaluator.metricName: "rmse"})


40819.08762568456

In [27]:
df_transformed.lrModel()

AttributeError: ignored

In [ ]:
df=prediction.select('price','prediction').toPandas()

In [ ]:
df['residuals']= ((df['price']-df['prediction']))

In [ ]:
df['residuals'].sort_values()

In [ ]:
df['residuals'].hist(bins=50)

In [ ]:
df = prediction.select('bedrooms','price','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','sqft_above','sqft_basement','yr_built','yr_renovated','street','city','statezip','prediction')

In [ ]:
df=df.toPandas()

In [ ]:
df.to_csv(cur_path+'prediction_data.csv')

In [ ]:
df.head()